# **Social Choice Course - Final Project**

#Introduction

A work by Lior Danino and Daniel Goman

This project features the algorithms suggested by M. Zuckerman, A.D. Procaccia and J.S. Rosenchein in a paper they published in 2008.

Link to the paper: https://www.sciencedirect.com/science/article/pii/S0004370208001963

The paper discusses the topic of Constructive Coallitional Weighted/Unweighted Manipulation problem (CCWM/CCUM respectively) 

The paper features 3 algorithms that under some assumptions, correctly find if there's a coallitional manipulation in the Borda, Maximin and Plurality with Runoff voting rules.

The novelty of their work is presenting algorithms of polynomial time complexity in the size of the input to solve the problem of CCWM/CCUM, which is considered NP-Hard for a variety of prominent voting rules.

In our work we implement the 3 algorithms described in the paper, and run it on some demo input to display its validity.

In [95]:
import numpy as np
from operator import itemgetter
from itertools import product

**Borda rule:**

This is a weighted version of the Borda voting rule.

Each voter ranks the candidates with the scores *m-1, m-2,...,0* (*m* is the amount of candidates) where his favorite candidate gets *m-1* and his least favorite candidate gets *0*. Since the voters have weights, we multiply each score by the weight of the relevant voter.
The final Borda-score of each candidate is the sum of the scores (after multiplication in weights) given to him by all the voters.



---


Parameters:

        
        C: list, shape(num_candidates)
            set of candidates

        Ws: list, shape(num_voters)
            weights of truthful voters

        Wt: list, shape(num_manipulators) 
            weights of manipulators

        Prefs: list, shape(num_voters, num_candidates)
            preferences of voters

    
Returns:

        borda_scores: dict
            {candidate i: borda score}

In [96]:
# Borda

def borda_rule(C, Ws, Wt, prefs):
    m = len(C)
    scores = {i:[] for i in range(1, m+1)}
    W = Ws.copy()
    W.extend(Wt)
    for voter in range(len(prefs)):
        for i in range(m):
            scores[prefs[voter][i]].append((m-i-1) * W[voter])
    borda_scores = {i:sum(scores[i]) for i in range(1, m+1)}
    
    return borda_scores

**Maximin rule:**

This is a weighted version of the Maximin voting rule.

For any two distinct candidates x and y, the score for (x, y) is the sum of the weights of the voters who prefer x over y.
The final Maximin-score of each candidate x is the minimum score over all the (x, z) scores where z is every candidate different from x.



---


Parameters:

        
        C: list, shape(num_candidates)
            set of candidates

        Ws: list, shape(num_voters)
            weights of truthful voters

        Wt: list, shape(num_manipulators) 
            weights of manipulators

        Prefs: list, shape(num_voters, num_candidates)
            preferences of voters

    
Returns:

        maximin_scores: dict
            {candidate i: maximin score}

In [97]:
# Maximin

def maximin_rule(C, Ws, Wt, prefs):
    m = len(C)
    scores = {(i,j):[] for i, j in product(set(C), set(C)) if i != j}
    W = Ws.copy()
    W.extend(Wt)
    for voter in range(len(prefs)):
        for i in range(m-1):
            for j in range(m):
                if i == j or j < i:
                    continue
                scores[(prefs[voter][i], prefs[voter][j])].append(W[voter])

    # calc scores for each pair
    all_scores = {(i, j): sum(scores[(i, j)]) for i, j in product(set(C), set(C)) if i != j}
    # create an array with [candidate, score] for all the scores
    array_scores = np.array([[key[0], value] for key, value in all_scores.items()])
    # pick the minimal score for each candidate
    maximin_scores = {i: np.min(array_scores[array_scores[:, 0]==i][:, 1], axis=0) for i in range(1, m+1)}

    return maximin_scores

**get_scores:**

Calculates the score according to Borda or Maximin voting rules.


---



Parameters:

        rule: string
            name of the rule: 'Borda' or 'Maximin'

        C: list, shape(num_candidates)
            set of candidates

        Ws: list, shape(num_voters)
            weights of truthful voters

        Wt: list, shape(num_manipulators) 
            weights of manipulators

        Prefs: list, shape(num_voters, num_candidates)
            preferences of voters

    
Returns:

        scores: dict
            {candidate i: score}

In [98]:
def get_scores(rule, C, Ws, Wt, prefs):
    if rule == 'Borda':
        scores = borda_rule(C, Ws, Wt, prefs)
    else:
        scores = maximin_rule(C, Ws, Wt, prefs)
    return scores

**get_scores_i:**

return the score for a specific candidate according to unweighted Borda rule.

In [99]:
def get_score_i(rule, C, i):
    if rule == 'Borda':
        score = len(C) - i
    return score

# **Algorithm 1**

Checks whether there exists a manipulation in the weighted Borda and Maximin voting rules.

**Guarantees:**

• In the Maximin rule, if there exists a manipulation for an instance with certain weights, Algorithm 1 will succeed when given two copies of the set of manipulators.


---


Algorithm 1 is a general greedy algorithm.
Each manipulator ranks candidate p as his first preference.
Then, in a descending order according to their weights, each manipulator pick his next preferences to be the candidates who earn the least from being ranked next (the candidates who get the minimal score when being ranked as the next preference).

The algorithm return True if candidate p is the winner (got the highest score), and False otherwise.


---

Parameters:

        C: list, shape(num_candidates)
            set of candidates
        
        p: int
            the candidate that we manipulate for
        
        Xs: list, shape(num_voters, num_candidates)
            preferences of truthful voters
    
        Ws: list, shape(num_voters)
            weights of truthful voters
    
        Wt: list, shape(num_manipulators) 
            weights of manipulators
        
        rule: string
            name of the rule: 'Borda' or 'Maximin'
 
Returns:

        bool:
            True if manipulation exists
            False otherwise



In [100]:

def algorithm1(C, p, Xs, Wt, Ws, rule):
    # weights:  list of [weight, manipulator_id] items
    weights = [[weight, index] for (index, weight) in enumerate(Wt, 1)]
    # sorted_weights: weights sorted in descending order (by weights)
    sorted_weights = sorted(weights, key=itemgetter(0), reverse=True)
    # T_preferences: list of [[preference profile], manipulator_id] sorted by weights
    T_preferences = [[[] ,sorted_weights[i][1]] for i in range(len(Wt))]

    # all existing preferences --> updated after each manipulator
    preferences = Xs

    for i in range(len(Wt)):
        # put candidate p as first preference for all manipulators 
        T_preferences[i][0].append(p)

        # evaluate the score of each candidate (other than p) if manipulator i would put it as his next preference
        c_scores = {}
        # ignore candidate p
        candidates_to_check = [item for item in C if item != p]

        for k in range(len(C) - 1):
            # check different profiles for manipulator i
            for c in candidates_to_check:
                # use his preferences
                i_curr_pref = [T_preferences[i][0][j] for j in range(len(T_preferences[i][0]))]
                # put candidate c as next preference
                i_curr_pref.append(c)
                # put remaining candidates in preferences (order doesn't metter)
                remaining_c = [item for item in candidates_to_check if item != c]
                if remaining_c != []:
                    i_curr_pref.extend(remaining_c)
                profile = preferences.copy()
                profile.append(i_curr_pref)
                # get scores
                c_scores[c] = get_scores(rule, C, Ws, Wt, profile)[c]
            # get candidate with minimal score out of remaining candidates
            remaining_c_scores = {c: c_scores[c] for c in c_scores.keys() - T_preferences[i][0]}
            min_c = min(remaining_c_scores, key=remaining_c_scores.get)
            # put candidate with minimal score as next preference for manipulator i
            T_preferences[i][0].append(min_c)
            candidates_to_check = [item for item in candidates_to_check if item != min_c]
        # add manipulator i final profile to preferences
        preferences.append(T_preferences[i][0])
    # calculate scores 
    final_scores = get_scores(rule, C, Ws, Wt, preferences)
    max_c = max(final_scores, key=final_scores.get)

    return True if max_c == p else False

# **Algorithm 2**

Checks whether there exists a manipulation in the weighted Borda voting rule.

**Guarantees:**

• In the Borda rule, if there exists a manipulation for an instance with certain weights, Algorithm 2 will succeed when given an extra manipulator with maximal weight.


---


Each manipulator gives candidate p the maximal score. Since it is weighted, we multiply it by the weight of the manipulator. We then add it to the total Borda-score of p. The next step of the algorithm is to give the highest remaining score to the candidate with the lowest current score (and update his total Borda-score after multiplying by the relevant weight) until all the scores are given.

The algorithm return True if candidate p is the winner (got the highest score), and False otherwise.


---

Parameters:

        C: list, shape(num_candidates)
            set of candidates
        
        p: int
            the candidate that we manipulate for
        
        s_scores: dict {candidate: score}
            Borda scores based on truthful voters 
    
        Wt: list, shape(num_manipulators) 
            weights of manipulators
        
        rule: string
            name of the rule: 'Borda' 
 
Returns:

        bool:
            True if manipulation exists
            False otherwise



In [101]:
def algorithm2(C, p, s_scores, Wt, rule):
  # sorted_scores: sorted list of truthful voters scores [candidate, score]
  sorted_scores = sorted([[c, s] for c,s in s_scores.items()], key=itemgetter(1))
  
  # all existing scores --> updated after each manipulator
  scores = s_scores.copy()

  for i in range(len(Wt)):
    # update score for candidate p as first preference for manipulator i
    scores[p] = scores[p] + (get_score_i(rule, C, 1)  * Wt[i])
    # start building preferences for manipulator i
    i_preferences = [p]
    # sort candidates by score and add them to the profile in descending order
    sorted_scores = sorted([[c, s] for c,s in scores.items()], key=itemgetter(1))
    i_preferences.extend([c for c, s in sorted_scores if c != p])

    # update scores according to the preferences of manipulator i 
    for j, c in enumerate(i_preferences, 1):
      if c == p:
        continue
      scores[c] = scores[c] + (get_score_i(rule, C, j)  * Wt[i])
  
  max_c = max(scores, key=scores.get)

  return True if max_c == p else False
      

truthful_scores = get_scores('Borda', C, Ws, Wt, Xs)
algorithm_2 = algorithm2(C, p, truthful_scores, Wt, 'Borda')

**Algorithm 1 and Algorithm 2 comparison:**

kill me now

# **Algorithm 3**

Checks whether there exists a manipulation in the Plurality with Run-off voting rule.

**Guarantees:**

If there exists a manipulation for an instance with certain weights, Algorithm 3 will succeed when given an extra manipulator with maximal weight.

---

**Explanation of Algorithm 3:**


*   For every candidate (g) except our candidate of interest (p), we calculate the score increment they need to win the first round of the plurality with round-off

*   We check if the manipulators have enough weight to give this candidate the increment they need.
If they don't - continue to the next iteration (select a new (g) )

*   Now some manipulators vote for (g), and some for (p).

*   We use dynamic programming to obtain a partition over the set of manipulators into 2 subsets, one of which consists of manipulators that will vote for (p), and the oter concists of manipulators that will vote for (g). 
The partition maximizes the score of (p) while making sure (g) gets enough votes to pass to the next round.

*   We check if given this partition (p) does proceed to the second round (along with g which is guaranteed to).
If they don't - continue to the next iteration (select a new (g) )

*   If we reach this point, we have that both (p) and (g) proceed to the second round.

*   We calculate the pairwise scores between (p) and (g) (including the votes of the manipulators).

*   If (p) wins the pairwise match - the algorithm succeeded to find a manipulation of plurality with round-off in favor of (p) and returns True.
Otherwise, continue to the next iteration (select a new (g) )

*   If we failed for all candidates (g) - the algorithm couldn't find a manipulation, and returns False.

---

**Idea for extension:**

The algorithm returns only True/False, but in the way it was designed, it's quite simple to extend it to return the required votes of the group of manipulators to perform the manipulation it found:

Given the 'x' vector which is found during the dynamic programming, we have which manipulators vote for (p) and which vote for (g) in order to make the manipulation happen, so all it takes is to convert this vector into a set of preferences of the manipulators and return it back from the algorithm.















**plurality_scores_eval:**

Calculates the weighted plurality score for every candidate requested

---

Parameters:

        V: ndarray, shape(num_voters, num_candidates)
            set of preferences of some group of voters
    
        C: set, shape(num_candidates)
            set of candidates for which the plurality score is calculated
    
        W: list, shape(num_voters)
            weights of voters in V
    
Returns:

        scores: dict
            candidate: wighted plurality score

In [102]:
def plurality_scores_eval(V, C, W):
    scores = {candidate: 0 for candidate in C}
    for voter, candidate in enumerate(V[:, 0]):
        scores[candidate] += W[voter]

    return scores

**pairwise_scores_eval:**

Calculates the weighed pairwise scores between candidates

---

Parameters:

        V: ndarray, shape(num_voters, num_candidates)
            set of preferences of some group of voters

        C: set, shape(num_candidates)
            set of candidates for which the plurality score is calculated

        W: list, shape(num_voters)
            weights of voters in V
    
Returns:

        scores: dict
            (candidate i, candidate j): weighted count of pairwise victories of i over j

In [103]:
def pairwise_scores_eval(V, C, W):
    scores = {(i, j): 0 for i, j in product(C, C) if i != j}

    for voter, voter_pref in enumerate(V):
        winners = set()
        for c in voter_pref:
            for winner in winners:
                scores[(winner, c)] += W[voter]
            winners.add(c)

    return scores

**indices_wise:**

Checks if tie is broken in favor of 'first' using indices-wise tie breaking

---

Parameters:

        first: int
            candidate of interest (the one we're interested in checking whether he wins the tie break)

        second: int
            the adversary of 'first'
    
Returns:

        True - if the first-second pairwise match-up tie-breaks in favor of first (ignores actual scores)
        False - otherwise

In [104]:
def indices_wise(first, second):
    return first < second

**weight_subset_approximate:**

Uses dynamic programming to find the voting scheme for the manipulators to maximize candidate p's score, while getting some other selected candidate to pass to the second stage of the plurality with round-off.

---

Parameters:

        Wt: list, shape(num_manipulators)
            weights of manipulators

        required_inc: float
            the increment necessary for some selected candidate to pass to the second stage

        error: float
            size of the error window
    
Returns:

        1 - x_bar: ndarray, shape(num_manipulators)
            values of {0, 1}:
                0 - manipulators who place p first
                1 - manipulators who don't play p first

In [105]:
def weight_subset_approximate(Wt, required_inc, error):
    if not 0 <= error <= max(Wt):
        print(f'Error should be in range of [0, {max(Wt)}]')
        exit()

    voter_size = len(Wt)

    ku = np.floor(error / (2 * voter_size)) + 1

    # keeping a mapping to the original order before sorting by weight
    voter_mapping = [(weight, voter_id) for voter_id, weight in enumerate(Wt)]

    # sorting the weights for the dynamic programming algorithm
    weights = sorted(voter_mapping, key=lambda x: x[0])
    mapping = list(np.array(weights)[:, 1].reshape(-1))
    weights = list(np.array(weights)[:, 0].reshape(-1))

    # contribution of each  voter to the score we're trying to maximize (the score of candidate p)
    values = [np.floor(w / ku) for w in weights]

    # define the dynamic programming table:
    # shape is: (amount of voters, weight thresholds)
    # T[i, j]:
    #   maximal score for candidate p a subset of the i voters
    #   with the lowest weight, with a maximal weight of j
    max_score_not_p = np.sum(weights) - required_inc

    if max_score_not_p % voter_size == 0:
        # one to account for the last index, as the range is not inclusive
        col_size = (max_score_not_p / voter_size) + 1
    else:
        # one to account for the last index, as the range is not inclusive
        # one to account for the float division discrepancy
        col_size = (max_score_not_p // voter_size) + 1 + 1

    shape = (voter_size, int(col_size))
    T = np.empty(shape=shape)

    weight_inc = max_score_not_p // voter_size
    for i in range(shape[0]):
        for j in range(shape[1]):
            if i == 0:
                if weights[i] > j * weight_inc:
                    T[i, j] = 0
                else:
                    T[i, j] = values[i]
            else:
                if weights[i] > j * weight_inc:
                    T[i, j] = T[i - 1, j]
                else:
                    # adjusting index jump for the case where we use the i'th voter
                    if weights[i] % weight_inc == 0:
                        remaining_index = int(j - (weights[i] / weight_inc))
                    else:
                        remaining_index = int(j - (weights[i] // weight_inc) - 1)

                    # either use voter i to vote for p, or not
                    T[i, j] = max(
                        [values[i] + T[i - 1, remaining_index],
                         T[i - 1, j]])

    # Now we have the maximal score for p, that being in index T[shape[0] - 1, shape[1] - 1]
    # which follows from the definition of T.
    # All that remains is to find the voters that need to rank p first to get this score.

    # find indexes in T which are used to obtain the maximal value
    p_subset = []
    j = shape[1] - 1
    for i in reversed(range(shape[0])):
        if j == 0:  # found all voters who need to vote for p
            break

        if T[i, j] == T[i - 1, j]:  # voter i was not used to achieve the maxima
            continue
        else:  # voter i was used to achieve the maxima
            # append i to the subset of manipulators who vote p
            p_subset.append(i)

            # keeping the current score before starting to traverse backwards
            curr_score = T[i, j]

            # i was used -> need to reduce the current score by values[i]
            # -> traversing T back to the cell where T[i, ?] = current_score - i's value
            while T[i, j] > curr_score - values[i]:
                j = j - 1

    # set 1 for every manipulator that votes for p, 0 otherwise
    x_bar = np.zeros(shape=voter_size)
    for v in range(voter_size):
        if v in p_subset:
            # map each voter to its original index
            x_bar[mapping[v]] = 1

    return 1 - x_bar

**algorithm3**

---

Checks if given the parameters there exists a manipulation
that puts p in the first place under the plurality with
runoff voting rule.



Parameters:

        C: set, shape(num_candidates)
            set of candidates

        p: int
            the candidate that we manipulate for

        Xs: ndarray, shape(num_voters, num_candidates)
            preferences of voters in S

        Ws: list, shape(num_voters)
            weights of voters in S

        Wt: list, shape(num_manipulators 
            weights of manipulators

        u: float
            size of the error window

Returns:

        True - if a manipulation exists.
        False - otherwise

In [106]:
def algorithm3(C, p, Xs, Ws, Wt, u, tie_breaking):
    plurality_scores = plurality_scores_eval(Xs, C, Ws)
    pairwise_scores = pairwise_scores_eval(Xs, C, Ws)

    required_inc = {}

    plurality = plurality_scores.copy()
    del plurality[p]
    prev_g = -1
    for g in C - {p}:
        # excluding g
        del plurality[g]

        if prev_g != -1:
            # returning the previously excluded candidate to the group of candidates
            plurality[prev_g] = plurality_scores[prev_g]

        prev_g = g

        # finding the candidates that achieve the maximal plurality score
        max_score = np.array(list(plurality.values())).max()
        max_candidates = [candidate for candidate, score in plurality.items() if score == max_score]

        # assigning required increment in plurality of score for g to win the first round
        defeater_found = False
        for candidate in max_candidates:
            if tie_breaking(candidate, g):           # indices-wise tie breaking - smaller index wins

                required_inc[g] = max_score - plurality_scores[g] + 1
                defeater_found = True

        if not defeater_found:
            required_inc[g] = max_score - plurality_scores[g]

        # if the manipulators can't make g win the first round -> proceed to the next candidate
        if required_inc[g] > np.sum(np.array(Wt)):
            continue

        # dynamic programming
        # x_j = 0 -> votes for p
        # x_j = 1 -> votes for g
        if required_inc[g] > 0:
            x = weight_subset_approximate(Wt, required_inc[g], u)
        else:
            x = np.zeros(shape=len(Wt))

        winning_rival_found = False
        for c_rival in C - {p, g}:
            manipulation_inc = (1 - x) @ Wt

            # c_rival defeats p, either strong defeat or defeat by tie-break
            if plurality_scores[c_rival] > plurality_scores[p] + manipulation_inc or \
                    (plurality_scores[c_rival] == plurality_scores[p] + manipulation_inc and tie_breaking(c_rival, p)):
                # indices-wise tie breaking
                winning_rival_found = True
                break

        # if couldn't manage to make it so p also wins the first round while making g pass the first round
        # -> failed to make p win -> proceed to the next candidate
        if winning_rival_found:
            continue

        # pairwise scores including the group of manipulators
        p_over_g_score = pairwise_scores[(p, g)] + (1 - x) @ Wt
        g_over_p_score = pairwise_scores[(g, p)] + x @ Wt

        # p strongly wins or tied and tie breaks in their favor
        if p_over_g_score > g_over_p_score or \
                p_over_g_score == g_over_p_score and tie_breaking(p, g):
            # at this point we know that both p and g will proceed to the second round
            # and if the above condition holds, then p will beat g in the second round
            # thus a manipulation in favor of p is possible
            return True

    # couldn't find a way to get p to win
    return False

**Simulation to support guarantee for Algorithm 1:**

*   In the Maximin rule, if there exists a manipulation for an instance with certain weights, Algorithm 1 will succeed when given two copies of the set of manipulators.

---

Output explanation:

        True - Found a manipulation in favor of p
        False - Couldn't find a manipulation in favor or p


In [107]:
C = [1, 2, 3]
Xs = [[3, 1, 2], 
      [3, 1, 2], 
      [3, 1, 2], 
      [1, 2, 3], 
      [1, 2, 3], 
      [1, 2, 3]]
Ws = [3, 4, 2, 3, 2, 2]
Wt = [3, 2]
Wt_copies = [3, 2, 3, 2]

# for candidate (2) there is no possible manipulation, and we see that the algorithm
# indeed returns False
p = 2
alg_1_no_manipulation = algorithm1(C.copy(), p, Xs.copy(), Wt.copy(), Ws.copy(), 'Maximin')

# although candidate (1) isn't winning by maximin, under the given preferences
# there exists a manipulation in favor of (1)
# we give 2 copies of the manipulator set to Algorithm 1, which guarantees
# us it will find some manipulation
p = 1
alg_1_manipulation = algorithm1(C.copy(), p, Xs.copy(), Wt_copies.copy(), Ws.copy(), 'Maximin')

print("Output given input for which there is no manipulation: ", alg_1_no_manipulation)
print("Output given input for which there is a manipulation (with two copies of the manipulators): ", alg_1_manipulation)

Output for input in which there is no manipulation:  False
Output for input in which there is a manipulation (with two copies of the manipulators):  True


**Simulation to support guarantee for Algorithm 2:**

*   In the Borda rule, if there exists a manipulation for an instance with certain weights, Algorithm 2 will succeed when given an extra manipulator with maximal weight.

---

Output explanation:

        True - Found a manipulation in favor of p
        False - Couldn't find a manipulation in favor or p

In [108]:
C = [1, 2, 3]
p = 3
Xs = [[3, 1, 2], 
      [3, 1, 2], 
      [3, 1, 2], 
      [1, 2, 3], 
      [1, 2, 3], 
      [1, 2, 3]]
Ws = [3, 4, 2, 3, 2, 2]
Wt = [3, 2]
Wt_extended = [3, 2, 4]

truthful_scores = get_scores('Borda', C, Ws, Wt, Xs)
algorithm_2 = algorithm2(C, p, truthful_scores, Wt, 'Borda')

# There is no manipulation in favor of candidate (3) which would make them win.
p = 2
alg_2_no_manipulation = algorithm2(C.copy(), p, truthful_scores.copy(), Wt.copy(), 'Borda')

# although candidate (3) isn't winning by maximin, under the given preferences
# there exists a manipulation in favor of (3)
# we give 1 extra manipulator with maximal weight to Algorithm 2, which guarantees
# us it will find some manipulation
p = 3
alg_2_manipulation = algorithm2(C.copy(), p, truthful_scores.copy(), Wt_extended.copy(), 'Borda')

print("Output given input for which there is no manipulation:", alg_2_no_manipulation)
print("Output given input for which there is a manipulation (with extra manipulator with maximal weight): ", alg_2_manipulation)

Output for input in which there is no manipulation: False
Output for input in which there is a manipulation (with extra manipulator with maximal weight):  True


**Simulation to support guarantee for Algorithm 3:**

*   In the Plurality with Runoff rule, if there exists a manipulation for an instance with certain weights, Algorithm3will succeed whengiven an extra manipulator with maximal weight.

---

Output explanation:

        True - Found a manipulation in favor of p
        False - Couldn't find a manipulation in favor or p

In [109]:
V = [[1, 2, 3],
     [2, 1, 3],
     [3, 1, 2],]
V = np.array(V)

candidates = set([1, 2, 3])
p = 2               # manipulate in favor of candidate (2)  
Ws = [2, 1, 2]
Wt = [1, 1]
error = 1

# There's no manipulation in the above settings
alg_3_no_manipulation = algorithm3(C=candidates, p=p, Xs=V, Ws=Ws, Wt=Wt, u=error, tie_breaking=indices_wise)

Wt = [1, 1, 1]
# a manipulation exists in the new settings 

# Notice that the manipulation here is not strictly voting for (2), but
# to actually have (3) beat (1) so that p=(2) won't pairwise lose to (1).
# The algorithm needs to find this variation.

# add an extra manipulator with maximal weight (max weight = 1)
Wt += [max(Wt)]

alg_3_manipulation = algorithm3(C=candidates, p=p, Xs=V, Ws=Ws, Wt=Wt, u=error, tie_breaking=indices_wise)

print("Output given input for which there is no manipulation:", alg_3_no_manipulation)
print("Output given input for which there is a manipulation (with extra manipulator with maximal weight): ", alg_3_manipulation)

Output for input in which there is no manipulation: False
Output for input in which there is a manipulation (with extra manipulator with maximal weight):  True


# **Summary**

In this work we implemented the following algorithms:

*   Algorithm 1, which is guaranteed to find a manipulation (when there is one) in the Maximin rule when given two copies of the set of manipulators 
*   Algorithm 2, which is guaranteed to find a manipulation (when there is one) in the Borda rule when given an extra manipulator with maximal weight 
*   Algorithm 3, which is guaranteed to find a manipulation (when there is one) in the Plurality with Runoff rule when given an extra manipulator with maximal weight.

To show our implementation is valid, we ran those algorithms on demo input which consists of cases where there is and is no manipulation.

The paper features a proof to the time complexity of the algorithms. We had no intention of showing that, as our work was around implementation of the algorithms featured in the paper.

For the purposes of testing the validity of those algorithms there was no need to use large sets of voters and preferences, and small examples which we could confirm sufficed.